In [34]:
import tensorflow
import torch

from tensorflow import keras
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt

from PIL import Image

import os
import random
import pandas as pd
import torchvision
import torch
import torchvision.transforms as transforms
from torchvision.models import vision_transformer
from torch.utils.data import Dataset, DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor
import torch.nn.functional as F
from pytorch_pretrained_vit import ViT
from timm import create_model
import torch.nn as nn
from tqdm import tqdm

In [35]:
path = '../../objrecognition/objrecognition/norm'

files = os.listdir(path)
dataset = []

for file in files:
    file_path = os.path.join(path, file)
    if os.path.isdir(file_path):
        images = os.listdir(file_path)
        for image in images:
            img = Image.open(os.path.join(file_path, image))
            img = img.convert("RGB")
            img = img.copy()
            dataset.append([img, int(file.split()[0][1])])
print(len(dataset))

1281


In [36]:
random.shuffle(dataset)
train_dataset = dataset[0:1001]
val_dataset = dataset[1001:1141]
test_dataset = dataset[1141:1281]
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

1001
140
140


In [37]:
train_dataset_x = []
train_dataset_y = []
val_dataset_x = []
val_dataset_y = []
test_dataset_x = []
test_dataset_y = []

for i in range(len(train_dataset)):
    train_dataset_x.append(train_dataset[i][0])
    train_dataset_y.append(train_dataset[i][1])
for i in range(len(val_dataset)):
    val_dataset_x.append(val_dataset[i][0])
    val_dataset_y.append(val_dataset[i][1])
for i in range(len(test_dataset)):
    test_dataset_x.append(test_dataset[i][0])
    test_dataset_y.append(test_dataset[i][1])

In [38]:
class CustomDataset(Dataset):
    def __init__(self, data, labels, img_transform):
        self.data = data
        self.labels = labels
        self.img_transform = img_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        sample = self.img_transform(sample)
        label = self.labels[idx]
        return sample, label

In [39]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = CustomDataset(train_dataset_x, train_dataset_y, transform)
val_dataset = CustomDataset(val_dataset_x, val_dataset_y, transform)
test_dataset = CustomDataset(test_dataset_x, test_dataset_y, transform)

batch_size = 16

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [40]:
model = create_model('vit_base_patch16_224', pretrained=True)
num_features = model.head.in_features
model.head = nn.Linear(num_features, 3)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30
for epoch in tqdm(range(num_epochs)):
    model.train()
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

model.eval()
total_accuracy = 0.0
total_samples = 0
for inputs, labels in val_dataloader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    predictions = F.softmax(outputs, dim=-1)
    predicted_classes = torch.argmax(predictions, dim=-1)
    accuracy = (predicted_classes == labels).float().mean()
    total_accuracy += accuracy.item() * inputs.size(0)
    total_samples += inputs.size(0)

print("Validation Accuracy:", total_accuracy / total_samples)

100%|██████████| 30/30 [04:55<00:00,  9.85s/it]


Validation Accuracy: 1.0


In [46]:
import matplotlib.pyplot as plt

for inputs, labels in val_dataloader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    predictions = torch.argmax(outputs, dim=-1)

    # Преобразуем тензоры меток в numpy массивы
    real_labels = labels.cpu().numpy()
    predicted_labels = predictions.cpu().numpy()

    # Добавляем метки в соответствующие списки
    real_labels_list.extend(real_labels)
    predicted_labels_list.extend(predicted_labels)

    # Показываем изображения
    for i in range(inputs.size(0)):
        img = inputs[i].permute(1, 2, 0).cpu().numpy()  # Переводим тензор изображения в массив numpy
        plt.imshow(img)
        plt.title(f'Real: {class_names[real_labels[i]]}, Predicted: {class_names[predicted_labels[i]]}')
        plt.axis('off')
        plt.show()


ValueError: too many values to unpack (expected 2)